## Wrappers. Frameworks. W&B Experiments Monitoring.

## 1. Wrappers

### Reward Wrapper
Мы знаем, что окружения создаются с помощью команды ``gym.make(<имя среды>)``, но что если мы хотим немного изменить окружение или добавить какой-то дополнительный функционал? Для этого существуют обертки (wrappers). Вспомним среду ``Taxi-v3``, предположим, что мы хотим поменять вознаграждения на следующие: 1 за решение задачи, -1 за неправильную посадку/высадку пассажира и 0 во всех остальных случаях. Для этого мы можем воспользоваться ``RewardWrapper``-ом:

In [ ]:
import gym

try:
    import colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

In [ ]:
gym.__version__

'0.17.3'

In [ ]:
class MyRewardWrapper(gym.RewardWrapper):
    def reward(self, reward):
        if reward == -1:
            return 0
        elif reward == 20:
            return 1
        elif reward == -10:
            return -1
        else: 
            raise KeyError

In [ ]:
env = gym.make("Taxi-v3")
env = MyRewardWrapper(env)
observation = env.reset()

rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if done:
        break

# выведем все вознаграждения, которые получал агент
print(rewards)
env.close()

{0, -1}


### Time Limit Wrapper

В зависимости от рандома, мы могли получить разные результаты, но обычно это ``{0, -1}. Откуда такой результат? Ведь среда заканчивается только когда задание выполнено. Все дело во встроенной обертке ограничивающей максимальное количество шагов. 

In [ ]:
env = gym.make("Taxi-v3")
print(type(env))
env.close()

<class 'gym.wrappers.time_limit.TimeLimit'>


In [ ]:
MyRewardWrapper(env)

<MyRewardWrapper<TimeLimit<TaxiEnv<Taxi-v3>>>>

Можно воспользоваться окружением без этой обертки, вызвав ``.env``:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)

observation = env.reset()
rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if done:
        break

print(rewards)
env.close()

{0, 1, -1}


А если у нас есть окружение без этого враппера по умолчанию, то можно добавить его вот так:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

observation = env.reset()
rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    print(observation)
    if done:
        break

print(rewards)
env.close()

143
{0}


Как вы считаете, корректно ли с точки зрения MDP, если окружения будет досрочно возвращать ``done=True``? Чтобы различать два случая завершения среды, в ``info`` передается дополнительный ключ ``TimeLimit.truncated: True``, если среду завершил ``TimeLimit`` враппер:

In [ ]:
env = gym.make("Taxi-v3")
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

env.reset()
_, _, done, info = env.step(env.action_space.sample())
print(f"done: {done}")
print(f"info: {info}")
env.close()

done: True
info: {'prob': 1.0, 'TimeLimit.truncated': True}


### Action Wrapper
Представим, что наш водитель находится не в лучшем своем состоянии и независимо от выбора агента, в 50% случаев совершает случайные действий. Сделать среду стохастическои и добиться такого эффекта мы можем, используя ``ActionWrapper``:

In [ ]:
import numpy as np

class TaxiRandomActionWrapper(gym.ActionWrapper):

    def __init__(self, env, probability=0.5):
        super().__init__(env) 
        self.probability = probability
        

    def action(self, action):
        if np.random.random() < self.probability:
            return env.action_space.sample()
        else: 
            return action

Чтобы проверить, что обертка работает будем выполнять единственное действие:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = TaxiRandomActionWrapper(env)

observation = env.reset()
rewards = set()
done = True
while done:
    action = 0
    observation, reward, done, info = env.step(action)
    rewards.add(reward)
    if done:
        break

print(rewards)
env.close()

{0}


### Wrapper

Класс ``gym.Wrapper`` является базовым для всех оберток. Подкласс может переопределить    многие методы для изменения поведения исходной среды, не изменяя при этом ее исходный код. Например, мы можем изменить метод step и добавить, какую-то дополнительную информацию в ``info``:

In [ ]:
class MyWrapper(gym.Wrapper):
    
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        info['Wrapped'] = True
        return observation, reward, done, info

In [ ]:
env = gym.make("Taxi-v3")
env = MyWrapper(env)

env.reset()
_, _, _, info = env.step(env.action_space.sample())

print(info)
env.close()

{'prob': 1.0, 'Wrapped': True}


### Monitor

Обертка ``Monitor``, позволяет сохранять часть эпизодов в видеоформате. Для ее работы в колабе нам придется настроить виртуальный дисплей.

In [ ]:
if COLAB:
    !wget https://gist.githubusercontent.com/Tviskaron/4d35eabce2e057dd2ea49a00b00aaa41/raw/f1e25fc6ac6d8f11cb585559ce8b2ab9ffefd67b/colab_render.sh -O colab_render.sh -q
    !sh colab_render.sh
    !wget https://gist.githubusercontent.com/Tviskaron/d91decc1ca5f1b09af2f9f080011a925/raw/0d3474f65b4aea533996ee00edf99a37e4da5561/colab_render.py -O colab_render.py -q 

    import colab_render

     |████████████████████████████████| 448 kB 33.8 MB/s 
Selecting previously unselected package python-opengl.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
(Reading database ... 157987 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.10_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.10) ...
Setting up xvfb (2:1.19.6-1ubuntu4.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 67 kB 5.8 MB/s 


In [ ]:
# библиотеки и функции, которые потребуеются показа видео

import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
from gym.envs.classic_control import rendering
import matplotlib.pyplot as plt

%matplotlib inline


org_constructor = rendering.Viewer.__init__


def constructor(self, *args, **kwargs):
    org_constructor(self, *args, **kwargs)
    self.window.set_visible(visible=False)


rendering.Viewer.__init__ = constructor


def show_video(folder="./video"):
    mp4list = glob.glob(folder + '/*.mp4')
    if len(mp4list) > 0:
        mp4 = sorted(mp4list, key=lambda x: x[-15:], reverse=True)[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

In [ ]:
if COLAB:
  !wget http://www.atarimania.com/roms/Roms.rar
  !mkdir /content/ROM/
  !unrar e /content/Roms.rar /content/ROM/
  !python -m atari_py.import_roms /content/ROM/

--2022-06-06 17:55:20--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19583716 (19M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  18.68M   820KB/s    in 24s     

2022-06-06 17:55:45 (798 KB/s) - ‘Roms.rar’ saved [19583716/19583716]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/128 in 1 Game Select ROM (Unknown) ~.bin          0%  OK 
Extracting  /content/ROM/2 in 1 - Chess, Othello (Atari) (Prototype).bin       0%  OK 
Extracting  /content/ROM/2 Pak Special - Cavern Blaster, City War (1992) (HES) (773-867) (PAL).bin       0%  OK 
Extracting  /content/ROM/2 Pak Special - Challenge, Surfing (1990) (HES) (771-333) (PAL).b

In [ ]:
!ls ./video

openaigym.episode_batch.0.71.stats.json
openaigym.manifest.0.71.manifest.json
openaigym.video.0.71.video000000.meta.json
openaigym.video.0.71.video000000.mp4


In [ ]:
env = gym.make("Breakout-v0")
env = gym.wrappers.Monitor(env, directory="./video", force=True)
# video_callable (Optional[function, False]): function that takes in the index of 
# the episode and outputs a boolean, indicating whether we should record a video on this episode. 
# The default (for video_callable is None) is to take perfect cubes, capped at 1000. 
# False disables video recording.
           
env.reset()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    if done:
        break

env.close()
show_video()

In [ ]:
# def capped_cubic_video_schedule(episode_id):
#   if episode_id < 1000:
#        return int(round(episode_id ** (1. / 3))) ** 3 == episode_id
#    else:
#        return episode_id % 1000 == 0

### Atari Preprocessing


Предварительная обработка для Atari 2600.
Класс ``AtariPreprocessing`` следует рекомендациям статьи: Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents" Machado et al. (2018). 

In [ ]:
env = gym.make("BoxingNoFrameskip-v0")

print(f'Original: {env.observation_space}')

Original: Box(0, 255, (210, 160, 3), uint8)


In [ ]:
env = gym.wrappers.AtariPreprocessing(gym.make("BoxingNoFrameskip-v0"))

print(f'Preprocessed: {env.observation_space}')

Preprocessed: Box(0, 255, (84, 84), uint8)


## 2. RL Frameworks

Переиспользование написанного кода является хорошей практикой написания программ и обучение с подкреплением не является исключением. Существует большое число готовых реализаций RL алгоритмов. Рассмотрим некоторые из них:

### OpenAI Baselines



<img src="https://github.com/openai/baselines/raw/master/data/logo.jpg" alt="Drawing" style="width: 200px;"/>

Цитата из [репозитория](https://github.com/openai/baselines):

«OpenAI Baselines is a set of high-quality implementations of reinforcement learning algorithms.

These algorithms will make it easier for the research community to replicate, refine, and identify new ideas, and will create good baselines to build research on top of. Our DQN implementation and its variants are roughly on par with the scores in published papers. We expect they will be used as a base around which new ideas can be added, and as a tool for comparing a new approach against existing ones.»

**Neural Network Framework:** Tensorflow 1.x

**Список алгоритмов:** A2C, ACER, ACKTR, DDPG, DQN, GAIL, HER, PPO, TRPO

Пример использования алгоритма DQN (без запуска, т.к. нужен tf1, а версия baselines с поддержкой tf2 еще в процессе): 
```python
import gym
from baselines import deepq


def main():
    env = gym.make("PongNoFrameskip-v4")
    env = deepq.wrap_atari_dqn(env)
    model = deepq.learn(
        env,
        "conv_only",
        convs=[(32, 8, 4), (64, 4, 2), (64, 3, 1)],
        hiddens=[256],
        dueling=True,
        total_timesteps=0
    )

    while True:
        obs, done = env.reset(), False
        episode_rew = 0
        while not done:
            env.render()
            obs, rew, done, _ = env.step(model(obs[None])[0])
            episode_rew += rew
        print("Episode reward", episode_rew)


if __name__ == '__main__':
    main()
```

Установка для colab и примеры визуализации: [baselines_viz.ipynb](https://colab.research.google.com/github/openai/baselines/blob/master/docs/viz/viz.ipynb).

In [ ]:
import numpy as np

q = np.array([1, 2])
q[None]

array([[1, 2]])

### Stable Baselines

<img src="https://miro.medium.com/max/1890/1*Qykf3HYCbSxw-nDYdLSbeQ.png" alt="Drawing" style="width: 500px;"/>


[Stable Baselines](https://github.com/hill-a/stable-baselines) $-$ это набор улучшенных реализаций алгоритмов обучения с подкреплением, основанный на OpenAI Baselines.

**Neural Network Framework:** Tensorflow 1.x

**Список алгоритмов:**  A2C, ACER, ACKTR, DDPG, DQN, GAIL, HER, **SAC**, **TD3**, PPO, TRPO

Запуск алгоритма PPO (Proximal Policy Optimization):
```python
# from https://github.com/hill-a/stable-baselines
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines import PPO2

env = gym.make('CartPole-v0')

model = PPO2(MlpPolicy, env, verbose=1)
# Train the agent
model.learn(total_timesteps=10000)

# Enjoy trained agent
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=False)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()
env.close()
```

Подробная статья о данном фреймворке: [stable-baselines-a-fork-of-openai-baselines-reinforcement)](https://towardsdatascience.com/stable-baselines-a-fork-of-openai-baselines-reinforcement-learning-made-easy-df87c4b2fc82)

### Stable Baselines3 


Stable Baselines3 $-$ это версия Stable Baselines на PyTorch.

**Neural Network Framework:** PyTorch.

**Список алгоритмов:** A2C, DDPG, DQN, HER, PPO, SAC, TD3

Tutorial: [Stable Baselines3 Tutorial](https://github.com/araffin/rl-tutorial-jnrr19)

In [ ]:
if COLAB:
    !pip install stable-baselines3[extra] --quiet

     |████████████████████████████████| 177 kB 29.0 MB/s 
     |████████████████████████████████| 1.5 MB 61.1 MB/s 
     |████████████████████████████████| 1.6 MB 60.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import gym
from stable_baselines3 import DQN

env = gym.wrappers.Monitor(gym.make('CartPole-v0'), directory='./video', force=True)
model = DQN('MlpPolicy', env, verbose=1, learning_starts=2000).learn(50000)
env.close()

show_video()

Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0573   |
|    n_updates        | 4746     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.4     |
|    ep_rew_mean      | 10.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1892     |
|    fps              | 701      |
|    time_elapsed     | 29       |
|    total_timesteps  | 21029    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.117    |
|    n_updates        | 4757     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.5     |
|    ep_rew_mean      | 10.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1896     |
|   

Изменим гиперпараметры и обучим с большим числом шагов:

In [ ]:
from stable_baselines3.common.monitor import Monitor

env = gym.wrappers.Monitor(gym.make('CartPole-v0'), directory='./video', force=True)
env = Monitor(env, filename='./stable-baselines-logs')

policy_kwargs = dict(net_arch=[200, 200])


model = DQN(policy='MlpPolicy', env=env, verbose=1, 
            learning_starts=2000, policy_kwargs=policy_kwargs,
            target_update_interval=2000)

model.learn(80000)

env.close()
show_video()

Using cuda device
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 18.8     |
|    ep_rew_mean      | 18.8     |
|    exploration_rate | 0.991    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 100      |
|    time_elapsed     | 0        |
|    total_timesteps  | 75       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 16.1     |
|    ep_rew_mean      | 16.1     |
|    exploration_rate | 0.985    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 137      |
|    time_elapsed     | 0        |
|    total_timesteps  | 129      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 16.2     |
|    ep_rew_mean      | 16.2     |
|    exploration_rate | 0.977    |
| 

Посмотрим на то, как проходило обучение:

In [ ]:
from stable_baselines3.common.monitor import load_results

print(load_results("."))

      index      r    l          t
0         0   12.0   12   0.376038
1         1   20.0   20   0.724613
2         2   29.0   29   0.777277
3         3   14.0   14   0.778665
4         4   11.0   11   0.785391
...     ...    ...  ...        ...
1052   1052  158.0  158  96.624453
1053   1053  144.0  144  96.792609
1054   1054  134.0  134  96.951248
1055   1055  123.0  123  97.091801
1056   1056  149.0  149  97.267467

[1057 rows x 4 columns]


### Rllib

<img src="https://docs.ray.io/en/master/_images/rllib-stack.svg" alt="Drawing" style="width: 600px;"/>


[RLlib](https://docs.ray.io/en/master/rllib.html) $-$ это open source RL фреймворк, который предлагает высокую масштабируемость и унифицированный API. RLlib изначально поддерживает TensorFlow, TensorFlow Eager и PyTorch. Для распараллеливания используется [Ray Project](https://github.com/ray-project/ray).

**Neural Network Framework:** Tensorflow, PyTorch.

**[Список алгоритмов:](https://docs.ray.io/en/master/rllib-algorithms.html)** A2C, A3C, ARS, BC, ES, DDPG, TD3, APEX-DDPG, Dreamer, DQN, Rainbow, APEX-DQN, IMPALA, MAML, MARWIL, MBMPO, PG, PPO, APPO, SAC, LinUCB, LinTS, AlphaZero, QMIX, MADDPG 


In [ ]:
# if COLAB:
if True:
    !pip install ray[rllib]==1.2.0 gym==0.17.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.5 MB 1.3 MB/s 
     |████████████████████████████████| 1.6 MB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.8 MB/s 
     |████████████████████████████████| 128 kB 59.4 MB/s 
     |████████████████████████████████| 244 kB 73.9 MB/s 
     |████████████████████████████████| 3.1 MB 50.1 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 71 kB 10.5 MB/s 
     |████████████████████████████████| 201 kB 71.7 MB/s 
     |████████████████████████████████| 125 kB 68.6 MB/s 
     |████████████████████████████████| 1.2 MB 61.0 MB/s 
     |████████████████████████████████| 21.6 MB 1.2 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 271 kB 71.7 MB/s 
     |██████████████████████████████

In [ ]:
import gym
gym.__version__

'0.17.3'

In [ ]:
import ray

ray.init(num_cpus=2, ignore_reinit_error=True, log_to_driver=False)

2022-06-06 18:14:19,060	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 61890,
 'node_id': 'b80b24182ac0841eebfd5762a973c2fa72e706b1a323745c50de7e81',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-06-06_18-14-18_388620_1336/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-06-06_18-14-18_388620_1336/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2022-06-06_18-14-18_388620_1336',
 'webui_url': '127.0.0.1:8265'}

In [ ]:
from ray.rllib.agents.dqn import DQNTrainer
from ray.tune import tune
import torch

def env_creator(env_config):
    pass

config = {"env": "CartPole-v0", "num_workers": 1, "monitor": True}

if torch.cuda.is_available():
    config['num_gpus'] = 1

results = tune.run("DQN",
                   config = config,
                   verbose=True,
                   stop={"timesteps_total": 10 ** 4},
                   checkpoint_freq=10)

Trial name,# failures,error file
DQN_CartPole-v0_c35f0_00000,1,/root/ray_results/DQN/DQN_CartPole-v0_c35f0_00000_0_2022-06-06_18-16-19/error.txt


TuneError: ignored

In [ ]:
run_folder = results.trials[0].logdir
show_video(folder=run_folder)

NameError: ignored

Загружаем сохраненные веса:

In [ ]:
policy = DQNTrainer(env="CartPole-v0", config=config)
policy.restore(run_folder + '/checkpoint_10/checkpoint-10')

2021-03-22 18:10:46,919	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-03-22 18:10:46,920	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-03-22 18:10:52,616	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-03-22 18:10:52,800	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-03-22 18:10:54,140	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2021-03-22 18:10:54,236	INFO trainable.py:372 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/DQN/DQN_CartPole-v0_82784_00000_0_2021-03-22_18-07-48/checkpoint_10/checkpoint-10
2021-03-22 18:10:54,237	INFO trainable.py:379 -- Current state after 

In [ ]:
env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, directory="./video", force=True)

obs = env.reset() 
done = False
while not done:
    obs, reward, done, _ = env.step(policy.compute_action(obs)) 
env.close()

show_video()

### PFRL

<img src="https://raw.githubusercontent.com/pfnet/pfrl/master/assets/PFRL.png" alt="Drawing" style="width: 180px;"/>


PFRL $-$ это библиотека глубокого обучения с подкреплением, которая реализует различные современные алгоритмы на Python, с использованием PyTorch. Бывший ChainerRL.

**Neural Network Framework:** PyTorch.

**Список алгоритмов:** DQN, Rainbow, IQN, DDPG, A3C, ACER, PPO, TRPO, TD3, SAC

Быстрый старт: [quickstart.ipynb](https://github.com/pfnet/pfrl/blob/master/examples/quickstart/quickstart.ipynb)

Примеры: [examples](https://github.com/pfnet/pfrl/tree/master/examples)

### Заслуживают упоминания:

* [OpenAI Spinning Up RL](https://github.com/openai/spinningup)
* [LeelaChessZero](https://github.com/LeelaChessZero)
* [Tianshou](https://github.com/thu-ml/tianshou)
* [TF agents](https://github.com/tensorflow/agents)
* [Catalyst-rl](https://github.com/catalyst-team/catalyst-rl)
* [Dopamine](https://github.com/google/dopamine)
* [TRFL](https://github.com/deepmind/trfl) 
* [Keras RL](https://github.com/keras-rl/keras-rl)
* [PyTorch-RL](https://github.com/Khrylx/PyTorch-RL)
* [PyMarl](https://github.com/oxwhirl/pymarl)
* [Denny Britz RL](https://github.com/dennybritz/reinforcement-learning)
* [DeepRL-Tutorials](https://github.com/qfettes/DeepRL-Tutorials)
* [RL Adventure](https://github.com/higgsfield/RL-Adventure)
* [FacebookResearch ELF](https://github.com/facebookresearch/ELF)
* [SLM-Lab](https://github.com/kengz/SLM-Lab)


## 3. Experiments monitoring. Weights & Biases.
Несколько примеров: [Dexterity](https://wandb.ai/openai/published-work/Learning-Dexterity-End-to-End--VmlldzoxMTUyMDQ), [RL Example: Pacman](https://wandb.ai/yashkotadia/rl-example).

Для того, чтобы результаты записывались в облако, нужно залогиниться. Предоставить свой api-key. Самый простой способ - использование консоли:

In [ ]:
if COLAB:
    !pip install wandb --quiet

import wandb
wandb.login()

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 163kB 23.7MB/s 
     |████████████████████████████████| 133kB 23.0MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import math
import random

# Start a new run, tracking hyperparameters in config
wandb.init(project="test-drive", config={
    "learning_rate": 0.01,
    "dropout": 0.2,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
})
config = wandb.config

# Simulating a training or evaluation loop
for x in range(50):
    acc = math.log(1 + x + random.random()*config.learning_rate) + random.random() + config.dropout
    loss = 10 - math.log(1 + x + random.random() + config.learning_rate*x) + random.random() + config.dropout
    # Log metrics from your script to W&B
    wandb.log({"acc":acc, "loss":loss})

wandb: Currently logged in as: tviskaron (use `wandb login --relogin` to force relogin)
